# Setup Training Data for 10-Minute GraphSAGE Training

Hướng dẫn tạo dữ liệu lớn hơn để train model trong ~10 phút.

**Cấu trúc dữ liệu mục tiêu:**
- 2000 samples training (thay vì 240)
- Validate/test proportional
- Kéo dài training ~10 phút với GPU

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import networkx as nx
from pathlib import Path
from tqdm import tqdm
import torch

# Thêm project path
project_root = Path('../').resolve()
sys.path.insert(0, str(project_root / 'src'))

print(f"✓ Project root: {project_root}")
print(f"✓ Python version: {sys.version}")

## 1. Define Project Structure

In [ ]:
# Định nghĩa cấu trúc thư mục
DATA_DIR = project_root / 'data' / 'graph_medium'
MODELS_DIR = project_root / 'models'
LOGS_DIR = project_root / 'logs'

# Tạo thư mục nếu chưa có
for dir_path in [DATA_DIR, MODELS_DIR, LOGS_DIR]:
    os.makedirs(dir_path, exist_ok=True)
    print(f"✓ Created/verified: {dir_path}")

print("\n📁 Project Structure:")
print(f"   DATA_DIR   = {DATA_DIR}")
print(f"   MODELS_DIR = {MODELS_DIR}")
print(f"   LOGS_DIR   = {LOGS_DIR}")

## 2. Load Graph Data

In [ ]:
# Kiểm tra và load dữ liệu graph
nodes_file = DATA_DIR / 'graph_nodes.csv'
edges_file = DATA_DIR / 'graph_edges.csv'

if not nodes_file.exists():
    print(f"⚠️  {nodes_file} không tồn tại. Tôi sẽ sinh dữ liệu graph...")
    # Sinh graph mới nếu chưa có
    import subprocess
    result = subprocess.run([sys.executable, str(project_root / 'src' / 'data' / 'generate_graph.py')],
                          capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")

# Load dữ liệu
nodes_df = pd.read_csv(nodes_file)
edges_df = pd.read_csv(edges_file)

print(f"\n📊 Graph Statistics:")
print(f"   Nodes: {len(nodes_df)}")
print(f"   Edges: {len(edges_df)}")
print(f"\n   Nodes sample:\n{nodes_df.head()}")
print(f"\n   Edges sample:\n{edges_df.head()}")

## 3. Build NetworkX Graph & Generate Path Samples

In [ ]:
# Build NetworkX graph để tính shortest path nhanh
print("🔨 Building NetworkX graph...")
G = nx.Graph()
num_nodes = len(nodes_df)
G.add_nodes_from(range(num_nodes))

for _, row in edges_df.iterrows():
    src, dst, weight = int(row['source']), int(row['target']), float(row['weight'])
    if src < num_nodes and dst < num_nodes:
        G.add_edge(src, dst, weight=weight)

print(f"   ✓ Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Sinh dữ liệu training samples
print("\n🎯 Generating 2000 path samples for 10-minute training...")
NUM_SAMPLES = 2000
np.random.seed(42)

sources = []
targets = []
costs = []

# Random select source-target pairs
for i in tqdm(range(NUM_SAMPLES), desc="Generating paths"):
    src = np.random.randint(0, num_nodes)
    dst = np.random.randint(0, num_nodes)
    
    if src == dst:
        continue
    
    try:
        # Dùng NetworkX để tính chi phí Dijkstra nhanh
        cost = nx.shortest_path_length(G, source=src, target=dst, weight='weight')
        sources.append(src)
        targets.append(dst)
        costs.append(cost)
    except nx.NetworkXNoPath:
        continue

# Đảm bảo có đủ samples
sources = sources[:NUM_SAMPLES]
targets = targets[:NUM_SAMPLES]
costs = costs[:NUM_SAMPLES]

print(f"   ✓ Generated {len(sources)} valid paths")
print(f"   Cost statistics: min={min(costs):.2f}, max={max(costs):.2f}, mean={np.mean(costs):.2f}")

## 4. Split Data into Train/Val/Test

In [ ]:
# Tạo DataFrame
paths_df = pd.DataFrame({
    'source': sources,
    'target': targets,
    'cost': costs
})

# Split: Train 80% (1600), Val 10% (200), Test 10% (200)
n_total = len(paths_df)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)

train_df = paths_df[:n_train]
val_df = paths_df[n_train:n_train + n_val]
test_df = paths_df[n_train + n_val:]

# Lưu CSV
train_file = DATA_DIR / 'paths_train.csv'
val_file = DATA_DIR / 'paths_val.csv'
test_file = DATA_DIR / 'paths_test.csv'

train_df.to_csv(train_file, index=False)
val_df.to_csv(val_file, index=False)
test_df.to_csv(test_file, index=False)

print(f"💾 Dataset Split:")
print(f"   Training:   {len(train_df):4d} samples → {train_file}")
print(f"   Validation: {len(val_df):4d} samples → {val_file}")
print(f"   Test:       {len(test_df):4d} samples → {test_file}")
print(f"   Total:      {n_total:4d} samples")

## 5. Setup Training Configuration for 10-Minute Training

In [ ]:
# Cấu hình training để chạy ~10 phút
config = {
    "model": {
        "hidden_dim": 64,
        "num_layers": 2,
        "dropout": 0.3,
        "aggregator": "mean"
    },
    "training": {
        "num_epochs": 100,           # 10 phút / 100 epochs ≈ 6 giây/epoch
        "batch_size": 32,
        "learning_rate": 0.001,
        "weight_decay": 1e-5,
        "optimizer": "adam"
    },
    "data": {
        "num_train_samples": len(train_df),
        "num_val_samples": len(val_df),
        "num_test_samples": len(test_df),
        "num_nodes": num_nodes,
        "num_edges": G.number_of_edges()
    },
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

# Lưu config
config_file = MODELS_DIR / 'config.json'
with open(config_file, 'w') as f:
    json.dump(config, f, indent=2)

print("⚙️  Training Configuration:")
print(f"   Device: {config['device']}")
print(f"   Epochs: {config['training']['num_epochs']}")
print(f"   Batch size: {config['training']['batch_size']}")
print(f"   Learning rate: {config['training']['learning_rate']}")
print(f"   Estimated time: ~10 minutes")
print(f"\n   Config saved → {config_file}")

## 6. Project Structure Verification

In [ ]:
import os

def print_tree(directory, prefix="", max_depth=3, current_depth=0):
    """In cấu trúc thư mục"""
    if current_depth >= max_depth:
        return
    
    try:
        items = sorted(os.listdir(directory))
        items = [i for i in items if not i.startswith('.')]
        
        for i, item in enumerate(items):
            path = os.path.join(directory, item)
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            print(f"{prefix}{current_prefix}{item}")
            
            if os.path.isdir(path) and item not in ['.venv', '__pycache__', '.git']:
                extension = "    " if is_last else "│   "
                print_tree(path, prefix + extension, max_depth, current_depth + 1)
    except PermissionError:
        pass

print("📁 Project Structure Ready for Training:\n")
print_tree(str(project_root))

## 7. Next Steps - Start Training

Chạy lệnh sau để bắt đầu training trong 10 phút:

```bash
# Từ thư mục project root
python train_model.py --data-dir data/graph_medium --models-dir models --num-epochs 100 --batch-size 32

# Hoặc dùng script
./train.sh  # Trên Linux/Mac
train.bat   # Trên Windows
```

**Các file sẽ được sinh ra:**
- `models/best_model.pt` - Trọng số model (tự động lưu tốt nhất)
- `models/config.json` - Cấu hình training
- `models/results.json` - Kết quả (loss, metrics)
- `logs/training_*.log` - Chi tiết training